In [78]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import t
from datetime import datetime
from scipy.optimize import brentq
from sklearn.neighbors import KernelDensity
from scipy.optimize import minimize
import statsmodels.api as sm

In [79]:
df = pd.read_csv("/home/bethtian/fintech545/beth-fintech545/Project02/DailyPrices.csv")
df.set_index("Date", inplace = True)
df

,SPY,AAPL,NVDA,MSFT,AMZN,META,GOOGL,AVGO,TSLA,GOOG,...,SBUX,MMC,MDT,CB,LMT,KKR,MU,PLD,LRCX,EQIX
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,370.367920,123.768456,14.304595,236.183502,85.820000,124.265305,88.798103,53.397118,108.099998,89.378853,...,96.842888,161.264664,73.835571,214.210434,455.904877,45.874386,49.810677,106.143532,35.568180,639.647766
2023-01-04,373.227142,125.045052,14.738281,225.852097,85.139999,126.885315,87.761856,54.049290,113.639999,88.392403,...,100.329346,164.402908,76.392372,217.732574,454.921539,47.269882,53.598137,110.165482,36.268723,656.679016
2023-01-05,368.967438,123.718987,14.254632,219.158371,83.120003,126.456940,85.888649,53.545685,110.339996,86.459335,...,100.300537,161.420135,75.505508,216.917526,455.465698,46.581966,54.102486,106.294243,35.805130,637.822998
2023-01-06,377.428589,128.271118,14.848200,221.741211,86.080002,129.525223,87.024529,56.768929,113.059998,87.844360,...,102.471153,166.103210,76.288589,222.060059,451.809113,47.348503,56.139599,109.873489,38.226994,650.683350
2023-01-09,377.214661,128.795578,15.616640,223.900162,87.360001,128.977310,87.702065,55.655602,119.769997,88.482071,...,100.598274,165.549393,73.156288,216.869034,438.204468,48.783306,55.734154,109.317757,38.841690,657.654114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,595.010010,255.589996,137.009995,430.529999,223.750000,599.809998,192.759995,241.750000,431.660004,194.039993,...,92.250000,213.470001,80.739998,277.679993,488.970001,149.860001,88.514999,105.239998,73.400002,942.659973
2024-12-30,588.219971,252.199997,137.490005,424.829987,221.300003,591.239990,191.240005,235.580002,417.410004,192.690002,...,90.580002,211.979996,79.610001,275.410004,483.369995,148.389999,85.309998,105.099998,72.150002,935.929993
2024-12-31,586.080017,250.419998,134.289993,421.500000,219.389999,585.510010,189.300003,231.839996,403.839996,190.440002,...,91.250000,212.410004,79.879997,276.299988,485.940002,147.910004,84.160004,105.699997,72.230003,942.890015


In [80]:
required_stocks = ['SPY', 'AAPL', 'EQIX']
stock_prices = df[required_stocks]
portfolio = {
    'SPY': 100,  
    'AAPL': 200, 
    'EQIX': 150 
}

# A. 

In [81]:
def calculate_portfolio_value(prices, portfolio):
    total_value = 0
    for stock in portfolio:
        if stock in prices:
            total_value += prices[stock] * portfolio[stock]
    return total_value

target_date = '2025-01-03'
latest_prices = stock_prices.loc[target_date]

portfolio_value = calculate_portfolio_value(latest_prices, portfolio)
print(f"The total value of the investment portfolio: (Based on the latest price on {latest_prices.name}): ${portfolio_value:,.2f}")

print("\nThe composition of the investment portfolio:")
for stock in portfolio:
    if stock in latest_prices:
        stock_value = latest_prices[stock] * portfolio[stock]
        print(f"{stock}: {portfolio[stock]} shares x ${latest_prices[stock]:.2f} = ${stock_value:,.2f} ({stock_value/portfolio_value*100:.2f}%)")


The total value of the investment portfolio: (Based on the latest price on 2025-01-03): $251,862.50

The composition of the investment portfolio:
SPY: 100 shares x $591.95 = $59,195.00 (23.50%)
AAPL: 200 shares x $243.36 = $48,672.00 (19.32%)
EQIX: 150 shares x $959.97 = $143,995.50 (57.17%)


# B. 

In [82]:
returns = stock_prices.pct_change().dropna()
returns_zero_mean = returns - returns.mean()

position_values = {stock: latest_prices[stock] * portfolio[stock] for stock in portfolio if stock in latest_prices}

alpha = 0.05 
lambda_ewma = 0.97  
position_values

{'SPY': 59195.001220703125,
 'AAPL': 48672.00012207032,
 'EQIX': 143995.49560546875}

## B.a

In [93]:
def calculate_ewma_cov(returns, lambda_param):

    cov_matrix = returns.iloc[:20].cov()
    for t in range(20, len(returns)):
        r_t = returns.iloc[t].values.reshape(-1, 1)
        cov_matrix = lambda_param * cov_matrix + (1 - lambda_param) * (r_t @ r_t.T)
    
    return cov_matrix

sigma = calculate_ewma_cov(returns_zero_mean, lambda_ewma)

def calculate_var_es_normal(sigma, position_values, alpha):

    portfolio_positions = np.array([position_values.get(stock, 0) for stock in sigma.columns])
    portfolio_variance = portfolio_positions.T @ sigma.values @ portfolio_positions
    portfolio_volatility = np.sqrt(portfolio_variance)
    F_inverse_alpha = stats.norm.ppf(alpha)
    phi_F_inverse_alpha = stats.norm.pdf(F_inverse_alpha)
    var = -F_inverse_alpha * portfolio_volatility
    es = portfolio_volatility * phi_F_inverse_alpha / alpha
    
    return var, es

portfolio_var, portfolio_es = calculate_var_es_normal(sigma, position_values, alpha)

print(f"Portfolio VaR (5%): ${portfolio_var:,.2f}")
print(f"Portfolio ES (5%): ${portfolio_es:,.2f}")

print("\nRisk metrics for individual stocks:")
for stock in portfolio:
    if stock in sigma.columns:
        stock_volatility = np.sqrt(sigma.loc[stock, stock])
        stock_position = position_values.get(stock, 0)
        stock_var = -stats.norm.ppf(alpha) * stock_volatility * stock_position
        stock_es = stock_volatility * stats.norm.pdf(stats.norm.ppf(alpha)) / alpha * stock_position
        
        print(f"{stock} - VaR: ${stock_var:,.2f}, ES: ${stock_es:,.2f}")

Portfolio VaR (5%): $3,856.32
Portfolio ES (5%): $4,835.98

Risk metrics for individual stocks:
SPY - VaR: $827.85, ES: $1,038.16
AAPL - VaR: $946.08, ES: $1,186.42
EQIX - VaR: $2,933.51, ES: $3,678.74


## B.b

In [ ]:
def fit_t_distribution(x):
    def negative_log_likelihood(params):
        df, loc, scale = params
        return -np.sum(stats.t.logpdf(x, df=df, loc=loc, scale=scale))
    
    initial_guess = [5.0, np.mean(x), np.std(x)]
    bounds = [(2.0001, 50.0), (None, None), (1e-6, None)]
    
    result = minimize(negative_log_likelihood, initial_guess, bounds=bounds, method='L-BFGS-B')
    
    if result.success:
        df, loc, scale = result.x
        return df, loc, scale
    else:
        print("Warning: T-distribution fitting did not converge")
        return initial_guess[0], initial_guess[1], initial_guess[2]

t_params = {}
t_dists = {}

for stock in required_stocks:
    df_param, loc_param, scale_param = fit_t_distribution(returns[stock])
    t_params[stock] = (df_param, loc_param, scale_param)
    t_dists[stock] = stats.t(df=df_param, loc=loc_param, scale=scale_param)
    
    print(f"{stock} T-distribution parameters: df={df_param:.4f}, loc={loc_param:.6f}, scale={scale_param:.6f}")
U = pd.DataFrame()
for stock in required_stocks:
    U[stock] = t_dists[stock].cdf(returns[stock])

Z = pd.DataFrame()
for stock in required_stocks:
    Z[stock] = stats.norm.ppf(U[stock])

R = Z.corr(method='spearman')
print("\nCorrelation matrix (Spearman):")
print(R)

eigenvalues = np.linalg.eigvals(R)
if min(eigenvalues) < -1e-8:
    print("Warning: Correlation matrix is not positive definite")

    R = pd.DataFrame(
        np.linalg.matrix_power(
            np.dot(
                np.dot(
                    np.diag(np.diag(R) ** (-0.5)), 
                    np.dot(
                        np.dot(np.diag(np.diag(R) ** 0.5), R), 
                        np.diag(np.diag(R) ** 0.5)
                    )
                ), 
                np.diag(np.diag(R) ** (-0.5))
            ), 
            1
        ),
        index=R.index, columns=R.columns
    )

def simulate_t_copula(R, t_dists, n_sim=10000, seed=42):
    np.random.seed(seed)
    n_vars = len(R)
    L = np.linalg.cholesky(R)
    z = np.random.normal(0, 1, size=(n_sim, n_vars))
    x = z @ L.T
    u = stats.norm.cdf(x)
    sim_returns = {}
    for i, stock in enumerate(t_dists.keys()):
        sim_returns[stock] = t_dists[stock].ppf(u[:, i])
    
    return pd.DataFrame(sim_returns)

alpha = 0.05
n_sim = 10000

simulated_returns = simulate_t_copula(R.values, t_dists, n_sim)

simulated_values = pd.DataFrame(index=range(n_sim))

for stock in required_stocks:
    simulated_values[f"{stock}_value"] = position_values[stock] * (1 + simulated_returns[stock])

simulated_values["portfolio_value"] = simulated_values.sum(axis=1)
simulated_values["portfolio_pnl"] = simulated_values["portfolio_value"] - portfolio_value

sorted_pnl = simulated_values["portfolio_pnl"].sort_values()
var_index = int(n_sim * alpha)
portfolio_var = -sorted_pnl.iloc[var_index]
portfolio_es = -sorted_pnl.iloc[:var_index].mean()

print(f"\nPortfolio VaR (5%): ${portfolio_var:,.2f}")
print(f"Portfolio ES (5%): ${portfolio_es:,.2f}")

print("\nRisk metrics for individual stocks:")
for stock in required_stocks:
    stock_returns = simulated_returns[stock]
    stock_position = position_values[stock]

    stock_pnl = stock_position * stock_returns
    sorted_stock_pnl = stock_pnl.sort_values()
    stock_var = -sorted_stock_pnl.iloc[var_index]
    stock_es = -sorted_stock_pnl.iloc[:var_index].mean()
    print(f"{stock} - VaR: ${stock_var:,.2f}, ES: ${stock_es:,.2f}")


SPY T-distribution parameters: df=5.0000, loc=0.001139, scale=0.006611
AAPL T-distribution parameters: df=5.0000, loc=0.001428, scale=0.010530
EQIX T-distribution parameters: df=5.0000, loc=0.000786, scale=0.011735

Correlation matrix (Spearman):
           SPY      AAPL      EQIX
SPY   1.000000  0.624145  0.531030
AAPL  0.624145  1.000000  0.303198
EQIX  0.531030  0.303198  1.000000

Portfolio VaR (5%): $4,074.76
Portfolio ES (5%): $5,844.19

Risk metrics for individual stocks:
SPY - VaR: $722.88, ES: $1,080.40
AAPL - VaR: $971.75, ES: $1,384.71
EQIX - VaR: $3,216.40, ES: $4,739.73


## B.c

In [96]:
def calculate_historical_var(returns, holdings, current_prices, alpha=0.05, use_kde=False):

    valid_stocks = [stock for stock in holdings if stock in returns.columns and stock in current_prices]
    holdings_array = np.array([holdings[stock] for stock in valid_stocks])
    current_prices_array = np.array([current_prices[stock] for stock in valid_stocks])

    PV = np.sum(holdings_array * current_prices_array)
    returns_matrix = returns[valid_stocks].values
    sim_prices = (1 + returns_matrix) * current_prices_array.reshape(1, -1)
    portfolio_values = np.sum(sim_prices * holdings_array.reshape(1, -1), axis=1)
    portfolio_values.sort()
    
    n = len(portfolio_values)
    a = int(np.floor(alpha * n))
    VaR = PV - portfolio_values[a]

    #Using kde
    VaR_kde = None
    if use_kde:
        kde = KernelDensity(kernel='gaussian', bandwidth='scott').fit(portfolio_values.reshape(-1, 1))
        def quantile_kde(q):
            def f(x):
                log_density = kde.score_samples(np.array([[x]]))
                density = np.exp(log_density)
                points_below = portfolio_values[portfolio_values <= x]
                approx_cdf = len(points_below) / n
                
                return approx_cdf - q
            
            lower_bound = portfolio_values[0]
            upper_bound = portfolio_values[-1]
    
            return brentq(f, lower_bound, upper_bound)
        
        quantile = quantile_kde(alpha)
        VaR_kde = PV - quantile
    
    return PV, VaR, VaR_kde, portfolio_values

def calculate_es(portfolio_values, PV, VaR, alpha=0.05):
    threshold = PV - VaR
    shortfall_values = portfolio_values[portfolio_values <= threshold]
    ES = PV - np.mean(shortfall_values) if len(shortfall_values) > 0 else VaR
    return ES

In [97]:
current_prices = {}
for stock in required_stocks:
    current_prices[stock] = latest_prices[stock]

PV, VaR, VaR_kde, portfolio_values = calculate_historical_var(
    returns_zero_mean, portfolio, current_prices, alpha, use_kde=True
)

ES = calculate_es(portfolio_values, PV, VaR, alpha)

stock_metrics = {}
for stock in portfolio:
    if stock in returns_zero_mean.columns and stock in latest_prices:
        single_stock_holding = {stock: portfolio[stock]}

        stock_value, stock_var, stock_var_kde, stock_values = calculate_historical_var(
            returns_zero_mean, single_stock_holding, current_prices, alpha
        )
        stock_es = calculate_es(stock_values, stock_value, stock_var, alpha)
        
        stock_metrics[stock] = {"VaR": stock_var, "ES": stock_es}

print(f"Number of historical scenarios used: {len(portfolio_values)}")
print(f"Current portfolio value: ${PV:,.2f}")
print(f"Portfolio VaR (5%): ${VaR:,.2f}")

if VaR_kde is not None:
    print(f"Portfolio VaR with KDE smoothing (5%): ${VaR_kde:,.2f}")

print(f"Portfolio ES (5%): ${ES:,.2f}")

print("\nRisk metrics for individual stocks:")
for stock, metrics in stock_metrics.items():
    stock_value = portfolio[stock] * latest_prices[stock]
    var_pct = metrics["VaR"] / stock_value * 100
    es_pct = metrics["ES"] / stock_value * 100
    print(f"{stock} - VaR: ${metrics['VaR']:,.2f} ({var_pct:.2f}%), ES: ${metrics['ES']:,.2f} ({es_pct:.2f}%)")

var_pct = VaR / PV * 100
es_pct = ES / PV * 100
print("\nOverall risk metrics:")
print(f"Portfolio VaR: ${VaR:,.2f} ({var_pct:.2f}% of portfolio value)")
print(f"Portfolio ES: ${ES:,.2f} ({es_pct:.2f}% of portfolio value)")


Number of historical scenarios used: 503
Current portfolio value: $251,862.50
Portfolio VaR (5%): $4,577.27
Portfolio VaR with KDE smoothing (5%): $4,577.27
Portfolio ES (5%): $6,059.39

Risk metrics for individual stocks:
SPY - VaR: $872.43 (1.47%), ES: $1,080.10 (1.82%)
AAPL - VaR: $1,069.34 (2.20%), ES: $1,437.79 (2.95%)
EQIX - VaR: $3,650.99 (2.54%), ES: $4,714.89 (3.27%)

Overall risk metrics:
Portfolio VaR: $4,577.27 (1.82% of portfolio value)
Portfolio ES: $6,059.39 (2.41% of portfolio value)


# C.

In [103]:
# Method A: Normal distribution with EWMA covariance
normal_ewma_results = {
    'portfolio': {'VaR': 3856.32, 'ES': 4835.98},
    'stocks': {
        'SPY': {'VaR': 827.85, 'ES': 1038.16},
        'AAPL': {'VaR': 946.08, 'ES': 1186.42},
        'EQIX': {'VaR': 2933.51, 'ES': 3678.74}
    }
}

# Method B: T-distribution with Gaussian Copula
t_copula_results = {
    'portfolio': {'VaR': 4074.76, 'ES': 5844.19},
    'stocks': {
        'SPY': {'VaR': 722.88, 'ES': 1080.40},
        'AAPL': {'VaR': 971.75, 'ES': 1384.71},
        'EQIX': {'VaR': 3216.40, 'ES': 4739.73}
    }
}

# Method C: Historical Simulation
historical_results = {
    'portfolio': {'VaR': 4577.27, 'ES': 6059.39},
    'stocks': {
        'SPY': {'VaR': 872.43, 'ES': 1080.10},
        'AAPL': {'VaR': 1069.34, 'ES': 1437.79},
        'EQIX': {'VaR': 3650.99, 'ES': 4714.89}
    }
}

# Portfolio value and stock values
portfolio_value = 251862.50
stock_values = {
    'SPY': 59195.00,
    'AAPL': 48672.00,
    'EQIX': 143995.50
}


portfolio_df = pd.DataFrame({
    'Normal with EWMA': [
        normal_ewma_results['portfolio']['VaR'],
        normal_ewma_results['portfolio']['ES'],
        normal_ewma_results['portfolio']['VaR'] / portfolio_value * 100,
        normal_ewma_results['portfolio']['ES'] / portfolio_value * 100
    ],
    'T-dist with Copula': [
        t_copula_results['portfolio']['VaR'],
        t_copula_results['portfolio']['ES'],
        t_copula_results['portfolio']['VaR'] / portfolio_value * 100,
        t_copula_results['portfolio']['ES'] / portfolio_value * 100
    ],
    'Historical Simulation': [
        historical_results['portfolio']['VaR'],
        historical_results['portfolio']['ES'],
        historical_results['portfolio']['VaR'] / portfolio_value * 100,
        historical_results['portfolio']['ES'] / portfolio_value * 100
    ]
}, index=['VaR ($)', 'ES ($)', 'VaR (%)', 'ES (%)'])

stocks_df = {}
for stock in ['SPY', 'AAPL', 'EQIX']:
    stocks_df[stock] = pd.DataFrame({
        'Normal with EWMA': [
            normal_ewma_results['stocks'][stock]['VaR'],
            normal_ewma_results['stocks'][stock]['ES'],
            normal_ewma_results['stocks'][stock]['VaR'] / stock_values[stock] * 100,
            normal_ewma_results['stocks'][stock]['ES'] / stock_values[stock] * 100
        ],
        'T-dist with Copula': [
            t_copula_results['stocks'][stock]['VaR'],
            t_copula_results['stocks'][stock]['ES'],
            t_copula_results['stocks'][stock]['VaR'] / stock_values[stock] * 100,
            t_copula_results['stocks'][stock]['ES'] / stock_values[stock] * 100
        ],
        'Historical Simulation': [
            historical_results['stocks'][stock]['VaR'],
            historical_results['stocks'][stock]['ES'],
            historical_results['stocks'][stock]['VaR'] / stock_values[stock] * 100,
            historical_results['stocks'][stock]['ES'] / stock_values[stock] * 100
        ]
    }, index=['VaR ($)', 'ES ($)', 'VaR (%)', 'ES (%)'])
    
print("Portfolio Risk Metrics Comparison:")
print(portfolio_df)
print("\n")

for stock, df in stocks_df.items():
    print(f"{stock} Risk Metrics Comparison:")
    print(df)
    print("\n")

Portfolio Risk Metrics Comparison:
         Normal with EWMA  T-dist with Copula  Historical Simulation
VaR ($)       3856.320000         4074.760000            4577.270000
ES ($)        4835.980000         5844.190000            6059.390000
VaR (%)          1.531121            1.617851               1.817369
ES (%)           1.920087            2.320389               2.405833


SPY Risk Metrics Comparison:
         Normal with EWMA  T-dist with Copula  Historical Simulation
VaR ($)        827.850000          722.880000             872.430000
ES ($)        1038.160000         1080.400000            1080.100000
VaR (%)          1.398513            1.221184               1.473824
ES (%)           1.753797            1.825154               1.824647


AAPL Risk Metrics Comparison:
         Normal with EWMA  T-dist with Copula  Historical Simulation
VaR ($)        946.080000          971.750000            1069.340000
ES ($)        1186.420000         1384.710000            1437.790000
VaR (